# MuSyQLa

Analiza bazy danych dotyczącej muzyki _musicbrainz_ przy użyciu `python` (`pandas`, `seaborn`) oraz `SQL` (`postgreSQL`)

- [musicbrainz](musicbrainz);
- [python](https://www.python.org);
- [pandas](https://pandas.pydata.org);
- [postgreSQL](https://www.postgresql.org);

## SQL

**Instalacja środowiska SQL**:

1. zainstalowanie postgreSQL: `apt install postgresql` 
2. pobranie danych jako baza: `mbslave init --create-user --create-database` ([mbslave](https://github.com/acoustid/mbslave))

Poniżej znajduje się krótka konfiguracja SQL, coby go używać w pythonie i korzystać z ramek danych pandas.

In [1]:
try:

    def connect():
        import psycopg2
        return psycopg2.connect(
            host="localhost",
            database="musicbrainz",
            user="postgres",
            password="123"
        )
    
    CONNECTION = connect()

    print("Połączono z bazą danych")

    def SQL(q):
        from pandas import read_sql_query
        import warnings
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', category=UserWarning)
            
            # UserWarning: pandas only supports SQLAlchemy connectable 
            # (engine/connection) or database string URI or sqlite3 DBAPI2 connection. 
            # Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
            
            return read_sql_query(q, CONNECTION)
    
except Exception as e:
    print("Nie udało się połączyć z bazą danych. Błąd:", e)

Połączono z bazą danych


## Eksploracja

Tabel jest sporo co można zobaczyć zaglądając do `INFORMATION_SCHEMA.TABLES`

In [2]:
tables_count = SQL("""--sql

    SELECT COUNT(*) AS table_key 
        FROM INFORMATION_SCHEMA.TABLES
""")

tables_count

,table_key
0,703


Nie ma jednak co się zanużać w sprawdzaniu co i jak działa bo dokumentacja przedstawia dosyć klarownie jakie obiekty są w bazie, lista [_Core data_](https://musicbrainz.org/doc/MusicBrainz_Database):

> * `Areas` - Name, aliases, type, ISO codes, begin and end dates, disambiguation comment, MBID
> * `Artists` - Name, sort name, areas, IPI, ISNI, aliases, type, begin and end dates, disambiguation comment, MBID
> * `Events` - Name, aliases, type, begin and end dates, time, setlist, disambiguation comment, MBID (performers / location indicated through relationships)
> * `Genres` - Name, aliases, disambiguation comment, MBID
> * `Instruments` - Name, description, aliases, type, disambiguation comment, MBID
> * `Labels` - Name, area, IPI, ISNI, aliases, type, code, begin and end dates, disambiguation comment, MBID
> * `Mediums` - Format, list of tracks (title, artist credit, duration)
> * `Places` - Name, aliases, area, type, code, begin and end dates, disambiguation comment, MBID
> * `Recordings` - Title, artist credit, duration, ISRC, PUIDs, disambiguation comment, MBID
> * `Release Groups` - Title, artist credit, type, disambiguation comment, MBID
> * `Releases` - Title, artist credit, type, status, language, date, country, label, catalog number, barcode, medium(s), disc ID(s), ASIN, disambiguation comment, MBID
> * `Series` - Name, aliases, type, disambiguation comment, MBID (parts indicated through relationships)
> * `Works` - Title, ISWC, disambiguation comment, MBID (writers indicated through relationships)
> * `Relationships` & URLs - Relationships are a way to link the above entities together and allow MusicBrainz to capture most of the data contained in the liner notes of a CD.
> * `CD Stubs` - Title, artist, barcode, disc ID, disambiguation comment

Dokumentacja zapewnia też cały (ogromny) [schemat bazy](https://wiki.musicbrainz.org/images/5/5e/soup.svg). Na początek wystarczy schemat samego utworu ![](https://wiki.musicbrainz.org/images/9/9f/recording_entity_details.svg)

### Na przykładzie

Żeby się nie zagubić w tym gąszczu informacji poznawanie bazy zacznę od jakiegoś utworu muzycznego, bo do tego cały ten biznes się sprowadza. Żeby poziom wejścia był w miarę niski na początek zacznę od czegoś popularnego, a ostatnio zasłuchiwałem się do _Ain't Gonna Die Tonight_ Macklemora

<iframe style="border-radius:12px" src="https://open.spotify.com/embed/track/1FgmP7AOTcn7MGYmImj0kn?utm_source=generator" width="100%" height="352" frameBorder="0" allowfullscreen="" allow="autoplay; clipboard-write; encrypted-media; fullscreen; picture-in-picture" loading="lazy"></iframe>

In [40]:
track_search = SQL("""--sql

    SELECT * FROM musicbrainz.track 
        WHERE name LIKE 'Ain%t Gonna Die Tonight' LIMIT 10
""")

track_search

,id,gid,recording,medium,position,number,name,artist_credit,length,edits_pending,last_updated,is_data_track
0,23613235,00411bcc-0c15-4f2f-bf69-98922613e7ad,21374441,2164023,1,1,Ain’t Gonna Die Tonight,2056472,215000,0,2017-08-23 19:05:19.297020+00:00,False
1,23976093,691a5d82-14df-4739-9614-b5f4b48724ba,21374441,2200107,1,1,Ain’t Gonna Die Tonight,2056472,216000,0,2017-10-15 18:22:20.456296+00:00,False
2,25093866,43e9e0a1-bccb-49b9-a92b-ca6b726ef9a8,22429401,2310580,1,1,Ain’t Gonna Die Tonight,2056472,215000,0,2018-03-24 12:01:03.976779+00:00,False


Wyszukiwanie trwa dość długo, dodatkowo wyniki są 2 i póki co nie wiadomo czy poprawne, bo znalezienie artysty wymaga połączenia relacji między tabelami. Ze schematu wynika, że trzeba połączyć `track`, `artist_credit`, `artist_credit_name` i dopiero wtedy można znaleźć samego artystę w `artist`, chociaż jego imię może pojawi się w którejś ze wcześniejszych tabel.

In [42]:
track_search = SQL("""--sql

    SELECT * FROM musicbrainz.track AS tra
    LEFT JOIN musicbrainz.artist_credit AS cre 
        ON cre.id = tra.artist_credit
      
    WHERE tra.name LIKE 'Ain%t Gonna Die Tonight' LIMIT 10

""")

track_search

,id,gid,recording,medium,position,number,name,artist_credit,length,edits_pending,last_updated,is_data_track,id,name,artist_count,ref_count,created,edits_pending,gid
0,23613235,00411bcc-0c15-4f2f-bf69-98922613e7ad,21374441,2164023,1,1,Ain’t Gonna Die Tonight,2056472,215000,0,2017-08-23 19:05:19.297020+00:00,False,2056472,Macklemore feat. Eric Nally,2,5,2017-08-23 13:39:22.867031+00:00,0,f34bc17f-8c2b-33af-b26c-0dd61bac45d0
1,23976093,691a5d82-14df-4739-9614-b5f4b48724ba,21374441,2200107,1,1,Ain’t Gonna Die Tonight,2056472,216000,0,2017-10-15 18:22:20.456296+00:00,False,2056472,Macklemore feat. Eric Nally,2,5,2017-08-23 13:39:22.867031+00:00,0,f34bc17f-8c2b-33af-b26c-0dd61bac45d0
2,25093866,43e9e0a1-bccb-49b9-a92b-ca6b726ef9a8,22429401,2310580,1,1,Ain’t Gonna Die Tonight,2056472,215000,0,2018-03-24 12:01:03.976779+00:00,False,2056472,Macklemore feat. Eric Nally,2,5,2017-08-23 13:39:22.867031+00:00,0,f34bc17f-8c2b-33af-b26c-0dd61bac45d0


🗒 **Notatka SQLowa**: 

* trzeba pamiętać o kolejności - w skórcie ma być: _co_, _gdzie_, _jak_, więc `JOINy` są przed klauzulą `WHERE`, bo mówią co wziąć, a _where_ przecież mówi _gdzie_;
* możnaby rozważyć jak przyśpieszyć szybkość wyszukiwania, szczególnie w przypadku utworów popularnych;

#### Łączenie Relacji

Nazwa jest jako ciąg znaków, generalnie nie są to atomowe informacje, ale to jak podpisywany jest utwór. Żeby mieć informacje o konkretnym artyście trzeba iść dalej. Widać przynajmniej, że hipoteza, że dane są poprawne jest prawdziwa.

In [38]:
track_search = SQL("""--sql

    SELECT * FROM musicbrainz.track AS tra
    LEFT JOIN musicbrainz.artist_credit AS cre
        ON cre.id = tra.artist_credit
      
    WHERE tra.name LIKE 'Ain%t Gonna Die Tonight' LIMIT 10

""")

track_search

,id,gid,recording,medium,position,number,name,artist_credit,length,edits_pending,last_updated,is_data_track,id,name,artist_count,ref_count,created,edits_pending,gid
0,23613235,00411bcc-0c15-4f2f-bf69-98922613e7ad,21374441,2164023,1,1,Ain’t Gonna Die Tonight,2056472,215000,0,2017-08-23 19:05:19.297020+00:00,False,2056472,Macklemore feat. Eric Nally,2,5,2017-08-23 13:39:22.867031+00:00,0,f34bc17f-8c2b-33af-b26c-0dd61bac45d0
1,23976093,691a5d82-14df-4739-9614-b5f4b48724ba,21374441,2200107,1,1,Ain’t Gonna Die Tonight,2056472,216000,0,2017-10-15 18:22:20.456296+00:00,False,2056472,Macklemore feat. Eric Nally,2,5,2017-08-23 13:39:22.867031+00:00,0,f34bc17f-8c2b-33af-b26c-0dd61bac45d0
2,25093866,43e9e0a1-bccb-49b9-a92b-ca6b726ef9a8,22429401,2310580,1,1,Ain’t Gonna Die Tonight,2056472,215000,0,2018-03-24 12:01:03.976779+00:00,False,2056472,Macklemore feat. Eric Nally,2,5,2017-08-23 13:39:22.867031+00:00,0,f34bc17f-8c2b-33af-b26c-0dd61bac45d0


In [43]:
track_search = SQL("""--sql

    SELECT tra.*, art.name, art.id FROM musicbrainz.track AS tra

    LEFT JOIN musicbrainz.artist_credit AS cre 
        ON cre.id = tra.artist_credit
                                                 -- many2many - wiele nagrań ma różnych artystów
    LEFT JOIN musicbrainz.artist_credit_name AS cre_many2many 
        ON cre.id = cre_many2many.artist_credit
                   
    LEFT JOIN musicbrainz.artist AS art
        ON art.id = cre_many2many.artist
      
    WHERE tra.name LIKE 'Ain%t Gonna Die Tonight' LIMIT 10

""")

track_search

,id,gid,recording,medium,position,number,name,artist_credit,length,edits_pending,last_updated,is_data_track,name,id
0,23613235,00411bcc-0c15-4f2f-bf69-98922613e7ad,21374441,2164023,1,1,Ain’t Gonna Die Tonight,2056472,215000,0,2017-08-23 19:05:19.297020+00:00,False,Macklemore,309011
1,23613235,00411bcc-0c15-4f2f-bf69-98922613e7ad,21374441,2164023,1,1,Ain’t Gonna Die Tonight,2056472,215000,0,2017-08-23 19:05:19.297020+00:00,False,Eric Nally,1191421
2,23976093,691a5d82-14df-4739-9614-b5f4b48724ba,21374441,2200107,1,1,Ain’t Gonna Die Tonight,2056472,216000,0,2017-10-15 18:22:20.456296+00:00,False,Macklemore,309011
3,23976093,691a5d82-14df-4739-9614-b5f4b48724ba,21374441,2200107,1,1,Ain’t Gonna Die Tonight,2056472,216000,0,2017-10-15 18:22:20.456296+00:00,False,Eric Nally,1191421
4,25093866,43e9e0a1-bccb-49b9-a92b-ca6b726ef9a8,22429401,2310580,1,1,Ain’t Gonna Die Tonight,2056472,215000,0,2018-03-24 12:01:03.976779+00:00,False,Macklemore,309011
5,25093866,43e9e0a1-bccb-49b9-a92b-ca6b726ef9a8,22429401,2310580,1,1,Ain’t Gonna Die Tonight,2056472,215000,0,2018-03-24 12:01:03.976779+00:00,False,Eric Nally,1191421


🗒 **Notatka SQLowa**: 

* połączenie wielu `JOIN`ów to nic innego jak dopisanie kolejnego

#### Powtarzalna piosenka

Można by się jeszcze pokusić o jakieś kryterium wyboru jednego utworu spośród wielu, np. tego starszego, jako oryginał, ale do tego potrzebna jest nowa tablica `recording_first_release_date`

In [ ]:
'DROP VIEW IF EXISTS track_search_view;'

In [77]:
track_search = SQL("""--sql

    SELECT
        tra.id, tra.name, tra.length,
        art.name,
        dat.date_year AS year, dat.date_month AS month, dat.date_day AS day,
        rel.name AS release_name,
        are.name

        FROM musicbrainz.track AS tra

    LEFT JOIN musicbrainz.artist_credit AS cre 
        ON cre.id = tra.artist_credit
                                                 -- many2many - wiele nagrań ma różnych artystów
    LEFT JOIN musicbrainz.artist_credit_name AS cre_many2many 
        ON cre.id = cre_many2many.artist_credit
                   
    LEFT JOIN musicbrainz.artist AS art
        ON art.id = cre_many2many.artist
                   
    LEFT JOIN musicbrainz.medium AS med
        ON med.id = tra.medium

    LEFT JOIN musicbrainz.release AS rel
        ON rel.id = med.release
                   
    LEFT JOIN musicbrainz.release_country AS dat
        ON rel.id = dat.release
                   
    LEFT JOIN musicbrainz.country_area AS cou
        ON cou.area = dat.country
                   
    LEFT JOIN musicbrainz.area AS are
        ON are.id = cou.area
      
    WHERE tra.name LIKE 'Ain%t Gonna Die Tonight' LIMIT 10

""")

track_search

,id,name,length,name,year,month,day,release_name,name
0,23613235,Ain’t Gonna Die Tonight,215000,Macklemore,2017,9,22,GEMINI,United States
1,23613235,Ain’t Gonna Die Tonight,215000,Eric Nally,2017,9,22,GEMINI,United States
2,23976093,Ain’t Gonna Die Tonight,216000,Macklemore,2017,9,22,Gemini,United States
3,23976093,Ain’t Gonna Die Tonight,216000,Eric Nally,2017,9,22,Gemini,United States
4,25093866,Ain’t Gonna Die Tonight,215000,Macklemore,2017,9,22,Gemini,United States
5,25093866,Ain’t Gonna Die Tonight,215000,Eric Nally,2017,9,22,Gemini,United States


Tabel jest pogrom a i tak nie wszystkie jeszcze są tutaj połączone. Dodatkowo z jakiegoś powodu ta sama piosenka ma aż 3 rekordy. Mimo, że wszystkie są z tego samego dnia i kraju(!) Co najwyżej różnią się długością, ale i to nie koniecznie. Możliwe, że coś jeszcze kryje się w danych, ale fakt faktem jest taki, że do kompleksowej analizy trzebaby znaleźć coś co wybierało by najlepszy i prawdopodobny wynik szukania. 

🗒 **Notatka SQLowa**: 

* można wykorzystać składnie widoku do szybkiego tworzenia identycznych zapytań;

## Potencjalne zastosowania

- grafy...
- analiza popularności;
- predykcja, np. gatunku, języka;
- sianie hipotez, ale nie wiem jeszcze jakich;
